In [1]:
import nltk
from nltk import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymystem3
from pymystem3 import Mystem
import re
from bs4 import BeautifulSoup
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
folder = "/Учеба/infopoisk/task1" 
writeFolder = "/Учеба/infopoisk/task2" 

In [4]:
nltk.download("stopwords")
nltk.download('punkt') # без этого не работает word_tokenize
stop_ru = stopwords.words('russian') # союзы и тд
stop_en = stopwords.words('english')
m = Mystem()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [5]:
# регулярка для вычищения слов с цифрами или символами внутри (моне*та, например)
# не ищет слова, оканчивающиеся символом или цифрой (чтобы оставить все слова с точками и запятыми)
CLEANR = re.compile('[a-zA-Zа-яА-Я]*[^a-zA-Zа-яА-Я\s]+[a-zA-Zа-яА-Я]+')

In [ ]:
tokens = []
lemmas = {}

for dirpath, _, filenames in os.walk('/content/drive/My Drive/'+ folder):
  for filename in filenames:
    current_tokens = []
    current_lemmas = {}
    print(filename)
    with open('/content/drive/My Drive/'+ folder +'/' + filename, 'rb') as f:
        data = f.read()
        # очищает html тэги
        soup = BeautifulSoup(data, 'html.parser')
        for data in soup(['style', 'script']):
            # Remove tags
            data.decompose()
    
        # return data by retrieving the tag content
        answer = ' '.join(soup.stripped_strings)
      
        clean_answer = re.sub(CLEANR, ' ', answer)
        # убираем все цифры
        clean_answer = re.sub(r"\d+", " ", clean_answer)
        # удаляем знаки препинания
        clean_answer = re.sub(r'[^\w\s]', '', clean_answer)
        # выделяет токены - слова
        words = word_tokenize(clean_answer, language="russian") 

        with open('/content/drive/My Drive/' + writeFolder + '/tokens-' + filename.replace(".html", "")  + '.txt', 'w') as f:
          with open('/content/drive/My Drive/' + writeFolder + '/lemmas-' + filename.replace(".html", "")  + '.txt', 'w') as lemma_file:
            for word in words:
                word = word.lower()
                if word not in stop_ru and word not in stop_en:
                    if word not in current_tokens:
                        current_tokens.append(word)
                        f.write(f"{word}\n")   

                    if word not in tokens:
                        tokens.append(word)
                    
                    lemma = m.lemmatize(word)[0]
                    if lemma not in current_lemmas.keys():
                      current_lemmas[lemma] = set()
                    current_lemmas[lemma].add(word)  
                    if lemma not in lemmas.keys():
                      lemmas[lemma] = set()
                    lemmas[lemma].add(word)
                    
            for lemma in current_lemmas.keys():
              lemma_file.write(f"{lemma}: ")  
              for token in current_lemmas[lemma]:
                lemma_file.write(f"{token} ")
              lemma_file.write(f"\n")       

In [7]:
with open('/content/drive/My Drive/' + writeFolder + '/tokens.txt', 'w') as f:
  for token in tokens:
        f.write(f"{token}\n")       

with open('/content/drive/My Drive/' + writeFolder + '/lemmas.txt', 'w') as f:
  for lemma in lemmas.keys():
        f.write(f"{lemma}: ")  
        for token in lemmas[lemma]:
          f.write(f"{token} ")
        f.write(f"\n")       